In [1]:
# using Pickle, JSON, CSV, DataFrames, BenchmarkTools, Distributed, FileIO, Pickle, Unmarshal
# using NetPricing
# using JuMP
# using Gurobi

In [2]:
using JSON, Unmarshal
using JuMP, Gurobi
using SparseArrays


# Source-sink vector
# function sourcesink_vector(prob::AbstractProblem, o, d)
#     b = zeros(Int, nodes(prob))
#     b[o] = 1
#     b[d] = -1
#     return b
# end
function parse_problem_data(data)
    V = data["V"]
    A_data = data["A"]
    K_data = data["K"]

    K = length(K_data)
    b_k = zeros(Int, K, V)
    c = zeros(Float64, length(A_data))
    A = spzeros(Int, V, length(A_data))
    d_k = zeros(Float64, K)
    is_tolled = zeros(Bool, length(A_data))

    for (i, edge) in enumerate(A_data)
        src = edge["src"]
        dst = edge["dst"]
        cost = edge["cost"]
        toll = edge["toll"]

        c[i] = cost
        is_tolled[i] = toll
    end


    # Build incidence matrix
    for (i, arc) in enumerate(A_data)
        A[arc["src"], i] = 1
        A[arc["dst"], i] = -1
    end


    
    for (i, user) in enumerate(K_data)
        orig = user["orig"]
        dest = user["dest"]
        demand = user["demand"]

        b_k[i, orig] = 1
        b_k[i, dest] = -1
        d_k[i] = demand
    end

    return K, b_k, c, A, d_k, is_tolled
end


function load_json_data(file_path)
    data = JSON.parsefile(file_path)
    problem_data = data["problem"]
   return problem_data
end
K, b_k, c, A, d_k, is_tolled = parse_problem_data(load_json_data("../tmp/000000-000000-g40-05-P.json"));
# println(A[:,3])


In [8]:
# Define your parameters
# K = [number_of_users]
# b_k = [your_b_vector_for_each_user]  # b now depends on k
# c = [your_c_vector]
# A = [your_A_matrix]
# d_k = [your_demand_for_each_user]  # d now depends on k
# is_tolled = [your_binary_vector_indicating_whether_each_edge_is_tolled]


# Create a new model
model = Model(() -> Gurobi.Optimizer())
set_optimizer_attribute(model, MOI.NumberOfThreads(), 1)
set_optimizer_attribute(model, "TimeLimit", 3600) # stop the process after x seconds




# Define your variables
# @variable(model, t[i=1:length(c)]>=0)
@variable(model, t[i=1:length(c)], lower_bound=0)
@constraint(model, [i=1:length(c); !is_tolled[i]], t[i] == 0)

x = Dict()
l = Dict()
for k in 1:K
    x[k] = @variable(model, [1:length(c)], lower_bound=0, upper_bound=1, base_name="x[$k]")
    l[k] = @variable(model, [1:length(b_k[1,:])], base_name="l[$k]")
    # Define your constraints
    @constraint(model, A * x[k] .== b_k[k,:])
    @constraint(model, A' * l[k] .<= c + t)
    @constraint(model, (c+t)' * x[k] <= b_k[k,:]' * l[k])
end


# Define your objective
@objective(model, Max, sum(d_k[k] * (t' * x[k]) for k in 1:K))# objective remains the same

# Solve the model
optimize!(model)

# Print the results
println("Objective value: ", objective_value(model))

tvals = value.(model[:t]) 

xvals = Dict()
λvals = Dict()
for k in 1:K
    xvals[k] = value.(x[k])
    λvals[k] = value.(l[k])
end




Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-10
Set parameter Threads to value 1
Set parameter TimeLimit to value 3600
Set parameter Threads to value 1
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 24.04 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 1 threads

Optimize a model with 10538 rows, 10580 columns and 40334 nonzeros
Model fingerprint: 0x6e38c7f1
Model has 8034 quadratic objective terms
Model has 39 quadratic constraints
Variable types: 10374 continuous, 206 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 164 rows and 164 c

In [29]:
println("Problem info")

k=35
println()
println("USER k=",k)
# println("origin\t\t\t\t\t|\t",NetPricing.orig(prob_k))
# println("destination\t\t\t\t|\t",NetPricing.dest(prob_k))
println()
println("Dual objective b' λ[k]\t\t\t|\t", b_k[k,:]' * λvals[k])
println("Primal objective c' x[k]\t\t|\t", c' * xvals[k])
println("Profit from user b' λ[k] - c' x[k]\t|\t", b_k[k,:]' * λvals[k] - c' * xvals[k])
println("-----------------------------------------------------------\n")


println("λvals")
println("length\t\t\t\t\t|\t", length(λvals[k]))
println("vector\t\t\t\t\t|\t", λvals[k])
println()
# println("Vertices used index Vmap")
# println("length\t\t\t\t\t|\t", length(Vmap))
# println("vector\t\t\t\t\t|\t", Vmap)
println("-----------------------------------------------------------\n")

println("Arcs used xvals")
println("length\t\t\t\t\t|\t", length(xvals[k]))
println("vector\t\t\t\t\t|\t", xvals[k])
println()
println("Arcs used index")
println("Amap[xvals .== 1]\t\t\t|\t" , findall(xvals[k] .== 1.0))

println("-----------------------------------------------------------\n")

Problem info

USER k=35

Dual objective b' λ[k]			|	82.0
Primal objective c' x[k]		|	82.0
Profit from user b' λ[k] - c' x[k]	|	0.0
-----------------------------------------------------------

λvals
length					|	60
vector					|	[3.0, 3.0, 2.0, 3.0, 1.0, 2.0, 1.0, 0.0, 1.0, 3.0, 2.0, 2.0, -4.0, 0.0, 3.0, 0.0, -26.0, -13.0, 3.0, -6.0, -34.0, -27.0, -12.0, -6.0, -14.0, -52.0, -18.0, 0.0, 0.0, 8.0, -22.0, 9.0, 15.0, 28.0, 26.0, 23.0, 40.0, 36.0, 46.0, 61.0, 58.0, 80.0, 69.0, 69.0, 69.0, 48.0, 82.0, 86.0, 96.0, 82.0, 36.0, 32.0, 53.0, 75.0, 68.0, 26.0, 14.0, 27.0, 40.0, 33.0]

-----------------------------------------------------------

Arcs used xvals
length					|	206
vector					|	[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [5]:
M = [1 2 3; 4 5 6; 7 8 9]

# Generate a matrix of random integers between 0 and each component in M
random_matrix = rand.(M) .* M

3×3 Matrix{Vector{Float64}}:
 [0.495198]                                                       …  [0.91981, 2.97285, 1.44923]
 [2.90509, 0.236605, 0.411498, 2.98644]                              [1.23781, 2.13534, 3.78508, 4.70846, 0.109205, 3.01469]
 [1.38191, 4.86701, 3.41553, 4.73709, 1.90044, 4.26293, 1.65979]     [4.53214, 0.359557, 5.12434, 8.11995, 2.17101, 1.76057, 3.48199, 1.32179, 1.19732]

In [3]:
M = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Convert M to a matrix
M_matrix = hcat(M...)

# Generate a matrix of random integers between 0 and each component in M_matrix
random_matrix = rand.(0:M_matrix)

LoadError: MethodError: no method matching (::Colon)(::Int64, ::Matrix{Int64})

[0mClosest candidates are:
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T<:Real
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:50[24m[39m
[0m  (::Colon)(::A, ::Any, [91m::C[39m) where {A<:Real, C<:Real}
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:10[24m[39m
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:49[24m[39m
[0m  ...


In [13]:
M_matrix = [[1.2,3.4,5.6],[ 7.8,9.0,1.1]]
new_M = hcat(M_matrix...)'
new_M
#random_matrix = round.(Int, new_M)

2×3 adjoint(::Matrix{Float64}) with eltype Float64:
 1.2  3.4  5.6
 7.8  9.0  1.1

In [15]:
nested_array = [7.8, 9.0, 1.1]
matrix_array = hcat(nested_array...)'

3×1 adjoint(::Matrix{Float64}) with eltype Float64:
 7.8
 9.0
 1.1

In [ ]:
		println(size(MT_max))
		M_rnd = rand.(size(MT_max)) .* MT_max
		M_rnd = round.(Int, M_rnd)
		println(size(M_rnd))